In [1]:
'[local_postgres_Praxisprojekt_SQL]'
user = 'postgres'
password = 'Viatanoua32'
host = 'localhost'
port = 5432
database = 'Praxisprojekt_SQL'

In [20]:
import pandas as pd
import psycopg2
import configparser

config = configparser.ConfigParser()
config.read("config.ini")
params = config["local_postgres_Praxisprojekt_SQL"]
connection = psycopg2.connect(**params)
connection = psycopg2.connect(database="Praxisprojekt_SQL", user="postgres", password="Viatanoua32", host="localhost", port="5432")
cursor=connection.cursor()


In [16]:
cursor.execute("""select * from kunden;""")
for row in cursor.fetchall():
    print(row)

(1, 'Kirby Poxon', 'Friedrich-Wohler-Strasse 2, 64579 Gernsheim', '5235734980328510')
(2, 'Helen-elizabeth Liddell', 'Gerhardswinkel 1, 53572 Unkel', '5452530720864080')
(3, 'Neile Patinkin', 'Marktstrasse 30, 73207 Plochingen', '5518441858555480')
(4, 'Mannie Hendricks', 'Trierer Strasse 282, 56072 Koblenz', '5473450221585970')
(5, 'Orton Tye', 'Berliner Strasse 269, 65205 Wiesbaden', '5472347033020320')
(6, 'Bobbee McClaren', 'Potsdamer Strasse 46, 14469 Potsdam', '5270254082243370')
(7, 'Noelyn Sorel', 'Mummendorfer Weg 11, 23769 Fehmarn', '5157168399114020')
(8, 'See Dorman', 'Oester 1, 33428 Harsewinkel', '5132364984299640')
(9, 'Edy Delea', 'Burgplatz 2, 72764 Reutlingen', '5273849209977540')
(10, 'Ruy McCarry', 'Adenauerallee 209, 52066 Aachen', '5295286000737500')
(11, 'Toby Mancktelow', 'Carl-Benz-Strasse 21, 60386 Frankfurt am Main', '5137727097274260')
(12, 'Dorella Eles', 'Kaiserstrasse 73, 45699 Herten', '5152056017234780')
(13, 'Krissie Saunper', 'Heinrichstrasse 30, 0754

In [17]:
import psycopg2
import pandas as pd


# SQL-Abfrage ausführen und Daten in einen Pandas-DataFrame laden
query = "SELECT * FROM bestellung"
df = pd.read_sql(query, connection)

# Verbindung zur Datenbank schließen
connection.close()

# Den DataFrame anzeigen
print(df.head())

   bestellid  kundenid  produktid  menge bestelldatum
0          1        21         28      6   2022-09-30
1          2        14         18      7   2023-02-06
2          3         9         41      4   2022-12-16
3          4        13         14      8   2022-07-31
4          5         9         27      7   2023-02-05


C:\Users\User\AppData\Local\Temp\ipykernel_10368\2879934797.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [30]:
import plotly.express as px
r1 = pd.read_sql(query, connection)
fig1 = px.bar(r1, x="menge", y="bestellid", orientation='v', title='bestellung')
fig1.show()

C:\Users\User\AppData\Local\Temp\ipykernel_10368\3340032135.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [27]:
sql_command= """select produktid, max(preis) as max_payment
from produkt
join bestellung using (produktid)
group by produktid
order by max(menge) desc
limit 1;
"""
cursor.execute(sql_command)
for row in cursor.fetchall():
    print(row)

(36, Decimal('241'))


In [26]:
cursor.execute("""SELECT name, MAX(preis) AS max_payment
FROM produkt
GROUP BY name
ORDER BY max_payment DESC
LIMIT 1;
"""
)
for row in cursor.fetchall():
    print(row)


('David Yurman Cable Classic Armreif', Decimal('294'))


In [28]:
import psycopg2
import pandas as pd

# Verbindung zur PostgreSQL-Datenbank herstellen
verbindung = psycopg2.connect(
    host="localhost",
    port="5432",
    database="Praxisprojekt_SQL",
    user="postgres",
    password="Viatanoua32"
)

# Den Kunden abrufen, der am meisten bezahlt hat
abfrage_hoechste_zahlung = """
SELECT b.KundenID, k.Name, SUM(b.Menge * p.Preis) AS Gesamtbetrag
FROM bestellung b
JOIN kunden k ON b.KundenID = k.KundenID
JOIN produkt p ON b.ProduktID = p.ProduktID
GROUP BY b.KundenID, k.Name
ORDER BY Gesamtbetrag DESC
LIMIT 1
"""
df_hoechste_zahlung = pd.read_sql(abfrage_hoechste_zahlung, verbindung)

# Den Zeitraum der Bestellungen abrufen
abfrage_bestellungszeitraum = """
SELECT MIN(Bestelldatum) AS MinDatum, MAX(Bestelldatum) AS MaxDatum
FROM bestellung
"""
df_bestellungszeitraum = pd.read_sql(abfrage_bestellungszeitraum, verbindung)

# Den Wochentag abrufen, an dem am häufigsten eingekauft wird
abfrage_haeufigster_wochentag = """
SELECT EXTRACT(DOW FROM Bestelldatum) AS Wochentag, COUNT(*) AS Anzahl
FROM bestellung
GROUP BY Wochentag
ORDER BY Anzahl DESC
LIMIT 1
"""
df_haeufigster_wochentag = pd.read_sql(abfrage_haeufigster_wochentag, verbindung)

verbindung.close()

print("Kunde mit höchstem Gesamtbetrag:")
print(df_hoechste_zahlung)

print("\nZeitraum der Bestellungen:")
print("Von:", df_bestellungszeitraum['mindatum'].iloc[0])
print("Bis:", df_bestellungszeitraum['maxdatum'].iloc[0])

print("\nAm häufigsten eingekaufter Wochentag:")
wochentage = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
haeufigster_wochentag = int(df_haeufigster_wochentag['wochentag'].iloc[0])
print(wochentage[haeufigster_wochentag])

Kunde mit höchstem Gesamtbetrag:
   kundenid       name  gesamtbetrag
0         9  Edy Delea        8586.0

Zeitraum der Bestellungen:
Von: 2020-10-21
Bis: 2023-06-04

Am häufigsten eingekaufter Wochentag:
Donnerstag


C:\Users\User\AppData\Local\Temp\ipykernel_10368\3824889303.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_hoechste_zahlung = pd.read_sql(abfrage_hoechste_zahlung, verbindung)
C:\Users\User\AppData\Local\Temp\ipykernel_10368\3824889303.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_bestellungszeitraum = pd.read_sql(abfrage_bestellungszeitraum, verbindung)
C:\Users\User\AppData\Local\Temp\ipykernel_10368\3824889303.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_haeufigster_wochentag = pd.read_sql(a